In [4]:
## Flash unit
## Yicheng Hu, Victor M. Zavala 2017-08

using JuMP
using Ipopt

# define sets 
K = ["benzene","toluene","xylene"] # set of components
A = [15.9008; 16.0137; 16.1156]
A = Dict(zip(K,A));
B = [2788.51; 3096.52; 3395.57]
B = Dict(zip(K,B));
C = [-52.34; -53.67; -59.44];
C = Dict(zip(K,C));
key = "toluene"
Km = ["benzene","xylene"]

m = Model(solver = IpoptSolver(tol = 0.1, max_iter = 15000));

# variables
@variable(m, alpha[K]>=0)
@variable(m, P0[K]>=0)
@variable(m, ξ[K]>=0)
@variable(m, v[K]>=0)
@variable(m, l[K]>=0)
@variable(m,0<=x[K]<=1)
@variable(m,0<=y[K]<=1)

# data
   f = [30;50;40];  # split fraction for each component
   f = Dict(zip(K,f));       # assign to set

# data
T=390

# constraints
@constraint(m,ξ[key]==0.9)
@NLconstraint(m, [k in K], P0[k] == exp(A[k] - B[k]/(T + C[k])))
@NLconstraint(m, [k in K], alpha[k] == P0[k]/P0[key])
@NLconstraint(m, [k in Km], ξ[k] == alpha[k]*ξ[key]/(1+(alpha[k]-1)*ξ[key])); 
@NLconstraint(m, [k in K], v[k] == ξ[k]*f[k]);
@NLconstraint(m, [k in K], l[k] == (1-ξ[k])*f[k]);    
@NLconstraint(m, [k in K], x[k] == l[k]/sum(l[kk] for kk in K));
@NLconstraint(m, [k in K], y[k] == v[k]/sum(v[kk] for kk in K));

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       54
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       49

Total number of variables............................:       21
                     variables with only lower bounds:       15
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [5]:
println(getvalue(ξ))
println(getvalue(x))
println(getvalue(y))

ξ: 1 dimensions:
[benzene] = 0.9540331852350292
[toluene] = 0.9
[ xylene] = 0.7744726720812821
x: 1 dimensions:
[benzene] = 0.08954517577589817
[toluene] = 0.32467326785546047
[ xylene] = 0.5857815563686414
y: 1 dimensions:
[benzene] = 0.2736235396910204
[toluene] = 0.4302107263023754
[ xylene] = 0.2961657340066043
